In [11]:
import torch
from PIL import Image
from transformers import AutoProcessor, AutoModelForVision2Seq
import trimesh

# 1. Configure processor with explicit batch handling
processor = AutoProcessor.from_pretrained("HuggingFaceTB/SmolVLM-256M-Instruct")
processor.image_processor.size = {"longest_edge": 256}
processor.image_processor.max_image_size = {"longest_edge": 256}
processor.image_processor.do_image_splitting = False

# 2. Initialize model with MPS optimization
model = AutoModelForVision2Seq.from_pretrained(
    "HuggingFaceTB/SmolVLM-256M-Instruct",
    torch_dtype=torch.bfloat16,
    attn_implementation="eager"
).eval().to("mps")

# 3. Fixed processing function
def describe_3d_object(image_path):
    img = Image.open(image_path).convert("RGB")
    
    # Structure inputs correctly :cite[4]:cite[10]
    inputs = processor(
        text=["Describe the 3D structure of this object:"],  # Must be list
        images=[[img]],  # Nested list structure
        return_tensors="pt",
        truncation=True,
        max_length=100,
        padding="max_length"
    ).to("mps")
    
    # Stable generation parameters
    generated_ids = model.generate(
        **inputs,
        max_new_tokens=40,
        temperature=0.01,
        do_sample=False,
        pad_token_id=processor.tokenizer.eos_token_id
    )
    
    return processor.decode(generated_ids[0], skip_special_tokens=True)

# 4. Mesh generation remains unchanged
def text_to_mesh(description):
    desc = description.lower()
    if "chair" in desc:
        return trimesh.creation.box(extents=(0.5, 0.5, 1.0))
    elif "table" in desc:
        return trimesh.creation.cylinder(radius=0.3, height=1.2)
    else:
        return trimesh.creation.icosphere(subdivisions=2)

# Usage
desc = describe_3d_object("chair2.jpeg")
mesh = text_to_mesh(desc)
mesh.export("output.obj")

Some kwargs in processor config are unused and will not have any effect: image_seq_len. 


IndexError: list index out of range

In [15]:
import trimesh

def text_to_3d(text_description: str) -> trimesh.Trimesh:
    """Generate simple 3D geometry from text descriptions using keyword matching"""
    desc = text_description.lower()
    
    # Basic shape recognition from text
    if "chair" in desc:
        # Create chair-like geometry
        seat = trimesh.creation.box(extents=(0.5, 0.5, 0.1))
        legs = trimesh.creation.cylinder(radius=0.05, height=0.4)
        back = trimesh.creation.box(extents=(0.1, 0.5, 0.4))
        
        # Position components
        legs.apply_translation([0.2, 0.2, -0.2])
        back.apply_translation([0.25, 0, 0.2])
        return trimesh.util.concatenate([seat, legs, back])
    
    elif "table" in desc:
        # Create table geometry
        top = trimesh.creation.box(extents=(0.8, 0.8, 0.1))
        legs = trimesh.creation.cylinder(radius=0.07, height=0.7)
        
        # Position legs
        leg_positions = [[0.35, 0.35, -0.35], 
                       [-0.35, 0.35, -0.35],
                       [0.35, -0.35, -0.35],
                       [-0.35, -0.35, -0.35]]
        for pos in leg_positions:
            leg = legs.copy()
            leg.apply_translation(pos)
            top = trimesh.util.concatenate([top, leg])
        
        return top
    
    elif "sphere" in desc:
        return trimesh.creation.icosphere(subdivisions=3)
    
    elif any(word in desc for word in ["box", "cube", "block"]):
        return trimesh.creation.box(extents=(0.5, 0.5, 0.5))
    
    # Default to cylinder if no matches
    return trimesh.creation.cylinder(radius=0.3, height=1.0)

# Usage
mesh = text_to_3d("A wooden cabin with lots of details")
mesh.export("chair.obj")

# mesh = text_to_3d("Round coffee table with central pillar")
# mesh.export("table.obj")

'# https://github.com/mikedh/trimesh\nv 0.00000000 0.00000000 -0.50000000\nv 0.30000000 0.00000000 -0.50000000\nv 0.30000000 0.00000000 0.50000000\nv 0.00000000 0.00000000 0.50000000\nv 0.29423558 0.05852710 -0.50000000\nv 0.29423558 0.05852710 0.50000000\nv 0.27716386 0.11480503 -0.50000000\nv 0.27716386 0.11480503 0.50000000\nv 0.24944088 0.16667107 -0.50000000\nv 0.24944088 0.16667107 0.50000000\nv 0.21213203 0.21213203 -0.50000000\nv 0.21213203 0.21213203 0.50000000\nv 0.16667107 0.24944088 -0.50000000\nv 0.16667107 0.24944088 0.50000000\nv 0.11480503 0.27716386 -0.50000000\nv 0.11480503 0.27716386 0.50000000\nv 0.05852710 0.29423558 -0.50000000\nv 0.05852710 0.29423558 0.50000000\nv 0.00000000 0.30000000 -0.50000000\nv 0.00000000 0.30000000 0.50000000\nv -0.05852710 0.29423558 -0.50000000\nv -0.05852710 0.29423558 0.50000000\nv -0.11480503 0.27716386 -0.50000000\nv -0.11480503 0.27716386 0.50000000\nv -0.16667107 0.24944088 -0.50000000\nv -0.16667107 0.24944088 0.50000000\nv -0.21

In [18]:
import trimesh

def text_to_3d_basic(text: str) -> trimesh.Trimesh:
    """Simplified working version with basic shape reasoning"""
    text = text.lower()
    
    # Simple reasoning steps
    if "cabin" in text:
        # Base plate
        base = trimesh.creation.box([2.0, 1.5, 0.1])
        
        # Walls (hollow box)
        walls = trimesh.creation.box([1.8, 1.3, 2.0])  # Smaller than base
        walls.apply_translation([0.1, 0.1, 0.1])  # Center on base
        
        # Simple triangular roof
        roof = trimesh.Trimesh(
            vertices=[[0,0,0], [2,0,0], [1,1.5,1],  # Front triangle
                      [0,0,0], [2,0,0], [1,1.5,1]],  # Back triangle (duplicated)
            faces=[[0,1,2], [3,4,5]]  # Two triangles
        )
        roof.apply_translation([0, 0, 2.1])  # Place on top of walls
        
        return base + walls + roof
        
    elif "chair" in text:
        # Simple chair logic
        seat = trimesh.creation.box([0.5, 0.5, 0.1])
        legs = trimesh.creation.cylinder(radius=0.05, height=0.4)
        back = trimesh.creation.box([0.1, 0.5, 0.4])
        return seat + legs + back
        
    # Default to cube
    return trimesh.creation.box()

# Works with basic shapes
mesh = text_to_3d_basic("wooden cabin")
mesh.export("cabin.obj")

'# https://github.com/mikedh/trimesh\nv -1.00000000 -0.75000000 -0.05000000\nv -1.00000000 -0.75000000 0.05000000\nv -1.00000000 0.75000000 -0.05000000\nv -1.00000000 0.75000000 0.05000000\nv 1.00000000 -0.75000000 -0.05000000\nv 1.00000000 -0.75000000 0.05000000\nv 1.00000000 0.75000000 -0.05000000\nv 1.00000000 0.75000000 0.05000000\nv -0.80000000 -0.55000000 -0.90000000\nv -0.80000000 -0.55000000 1.10000000\nv -0.80000000 0.75000000 -0.90000000\nv -0.80000000 0.75000000 1.10000000\nv 1.00000000 -0.55000000 -0.90000000\nv 1.00000000 -0.55000000 1.10000000\nv 1.00000000 0.75000000 -0.90000000\nv 1.00000000 0.75000000 1.10000000\nv 0.00000000 0.00000000 2.10000000\nv 2.00000000 0.00000000 2.10000000\nv 1.00000000 1.50000000 3.10000000\nf 2 4 1\nf 5 2 1\nf 1 4 3\nf 3 5 1\nf 2 8 4\nf 6 2 5\nf 6 8 2\nf 4 8 3\nf 7 5 3\nf 3 8 7\nf 7 6 5\nf 8 6 7\nf 10 12 9\nf 13 10 9\nf 9 12 11\nf 11 13 9\nf 10 16 12\nf 14 10 13\nf 14 16 10\nf 12 16 11\nf 15 13 11\nf 11 16 15\nf 15 14 13\nf 16 14 15\nf 17 1

In [19]:
import trimesh
import numpy as np
from typing import Dict, List

class CoT3DGenerator:
    def __init__(self):
        self.components = []
        self.relationships = []
        self.default_params = {
            'width': 1.0,
            'depth': 1.0,
            'height': 1.0,
            'radius': 0.5,
            'position': (0, 0, 0),
            'rotation': (0, 0, 0)
        }

    def parse_description(self, text: str):
        """Step 1: Structural decomposition with basic reasoning"""
        text = text.lower()
        
        # Basic CoT parsing
        if "table" in text:
            self.components.append({
                'type': 'surface',
                'shape': 'round' if 'round' in text else 'rectangular',
                'position': 'top'
            })
            self._add_support_components(text)
            
        elif "chair" in text:
            self.components.append({'type': 'seat', 'position': 'base'})
            self.components.append({'type': 'backrest', 'position': 'rear'})
            self._add_legs(text)
            
        else:  # Generic object handling
            self.components.append({
                'type': 'main_body',
                'shape': self._detect_shape(text),
                'position': 'center'
            })

    def _detect_shape(self, text: str) -> str:
        """Shape reasoning from text"""
        if any(w in text for w in ['round', 'circular', 'cylindrical']):
            return 'cylinder'
        if any(w in text for w in ['angular', 'rectangular', 'boxy']):
            return 'box'
        if 'spherical' in text:
            return 'sphere'
        return 'box'

    def _add_support_components(self, text: str):
        """Reason about structural supports"""
        if 'legs' in text:
            count = 4 if 'four' in text else 1
            self.components.append({
                'type': 'leg',
                'count': count,
                'shape': 'cylinder',
                'position': 'under_surface'
            })
        elif 'pillar' in text:
            self.components.append({
                'type': 'central_support',
                'shape': 'cylinder',
                'position': 'center_bottom'
            })

    def generate_component(self, part: Dict) -> trimesh.Trimesh:
        """Step 2: Geometry generation with spatial reasoning"""
        params = self._calculate_dimensions(part)
        
        if part.get('shape', 'box') == 'cylinder':
            mesh = trimesh.creation.cylinder(
                radius=params['radius'],
                height=params['height']
            )
        elif part['shape'] == 'sphere':
            mesh = trimesh.creation.icosphere()
        else:
            mesh = trimesh.creation.box(
                extents=(params['width'], params['depth'], params['height'])
            )
            
        mesh.apply_translation(params['position'])
        return mesh

    def _calculate_dimensions(self, part: Dict) -> Dict:
        """Size reasoning based on component relationships"""
        params = self.default_params.copy()
        
        # Size hierarchy reasoning
        if part['type'] == 'surface':
            params.update({'width': 2.0, 'depth': 1.0, 'height': 0.1})
        elif part['type'] == 'leg':
            params.update({'radius': 0.05, 'height': 0.7})
        elif part['type'] == 'central_support':
            params.update({'radius': 0.15, 'height': 0.8})
            
        return params

    def assemble(self) -> trimesh.Trimesh:
        """Step 3: Spatial assembly with relationships"""
        full_mesh = None
        z_level = 0  # Track vertical positioning
        
        for part in self.components:
            mesh = self.generate_component(part)
            
            # Basic spatial reasoning
            if 'under' in part.get('position', ''):
                mesh.apply_translation([0, 0, z_level - mesh.extents[2]/2])
            elif 'top' in part.get('position', ''):
                z_level += mesh.extents[2]
                mesh.apply_translation([0, 0, z_level])
                
            if full_mesh:
                full_mesh += mesh
            else:
                full_mesh = mesh
                
        return full_mesh

    def generate(self, description: str) -> trimesh.Trimesh:
        """Full CoT pipeline"""
        self.parse_description(description)
        return self.assemble()

# Usage
generator = CoT3DGenerator()
mesh = generator.generate("A dining table with round top and four legs")
mesh.export("table.obj")

# Complex example
generator = CoT3DGenerator()
mesh = generator.generate("A lamp with cylindrical base and conical shade")
mesh.export("lamp.obj")

'# https://github.com/mikedh/trimesh\nv 0.00000000 0.00000000 -0.50000000\nv 0.50000000 0.00000000 -0.50000000\nv 0.50000000 0.00000000 0.50000000\nv 0.00000000 0.00000000 0.50000000\nv 0.49039264 0.09754516 -0.50000000\nv 0.49039264 0.09754516 0.50000000\nv 0.46193977 0.19134172 -0.50000000\nv 0.46193977 0.19134172 0.50000000\nv 0.41573481 0.27778512 -0.50000000\nv 0.41573481 0.27778512 0.50000000\nv 0.35355339 0.35355339 -0.50000000\nv 0.35355339 0.35355339 0.50000000\nv 0.27778512 0.41573481 -0.50000000\nv 0.27778512 0.41573481 0.50000000\nv 0.19134172 0.46193977 -0.50000000\nv 0.19134172 0.46193977 0.50000000\nv 0.09754516 0.49039264 -0.50000000\nv 0.09754516 0.49039264 0.50000000\nv 0.00000000 0.50000000 -0.50000000\nv 0.00000000 0.50000000 0.50000000\nv -0.09754516 0.49039264 -0.50000000\nv -0.09754516 0.49039264 0.50000000\nv -0.19134172 0.46193977 -0.50000000\nv -0.19134172 0.46193977 0.50000000\nv -0.27778512 0.41573481 -0.50000000\nv -0.27778512 0.41573481 0.50000000\nv -0.35

In [24]:
import trimesh
import numpy as np
from typing import Dict, List

class CoT3DGenerator:
    def __init__(self):
        self.components = []
        self.reasoning_log = []
        self.default_params = {
            'width': 1.0,
            'depth': 1.0,
            'height': 1.0,
            'radius': 0.5,
            'position': (0, 0, 0)
        }

    def log_step(self, message: str):
        """Generic reasoning logger"""
        self.reasoning_log.append(f"• {message}")

    def parse_description(self, text: str):
        """Generic structural decomposition"""
        self.log_step(f"Analyzing description: '{text}'")
        text = text.lower()

        # Generic object type detection
        main_obj = next((w for w in ['table', 'chair', 'lamp', 'shelf'] if w in text), 'object')
        self.log_step(f"Identified main object type: {main_obj[0] if main_obj[0] else 'generic object'}")

        # Shape detection
        shape = self._detect_shape(text)
        self.log_step(f"Detected primary shape: {shape} from keywords")
        
        # Main component
        self.components.append({
            'type': 'main_body',
            'shape': shape,
            'position': 'base'
        })

        # Support detection
        self._detect_supports(text)

    def _detect_shape(self, text: str) -> str:
        """Generic shape reasoning"""
        shape_keywords = {
            'round': 'cylinder',
            'circular': 'cylinder',
            'spherical': 'sphere',
            'rectangular': 'box',
            'angular': 'box',
            'conical': 'cone'
        }
        for kw, shape in shape_keywords.items():
            if kw in text:
                return shape
        return 'box'

    def _detect_supports(self, text: str):
        """Generic support reasoning"""
        support_types = {
            'legs': {'count': 4, 'shape': 'cylinder'},
            'pillar': {'count': 1, 'shape': 'cylinder'},
            'base': {'count': 1, 'shape': 'cylinder'},
            'arms': {'count': 2, 'shape': 'box'}
        }
        
        for kw, config in support_types.items():
            if kw in text:
                count = next((int(n) for n in text.split() if n.isdigit()), config['count'])
                self.log_step(f"Detected {count} {kw} from description")
                self.components.append({
                    'type': kw,
                    'shape': config['shape'],
                    'count': count,
                    'position': 'under_main' if kw == 'legs' else 'base'
                })

    def generate_component(self, part: Dict) -> trimesh.Trimesh:
        """Generic component generation"""
        self.log_step(f"Creating {part['type']} ({part['shape']})")
        params = self._calculate_dimensions(part)
        
        # Shape mapping
        shapes = {
            'box': trimesh.creation.box,
            'cylinder': lambda: trimesh.creation.cylinder(
                radius=params['radius'], height=params['height']),
            'sphere': trimesh.creation.icosphere,
            'cone': lambda: trimesh.creation.cone(
                radius=params['radius'], height=params['height'])
        }
        
        mesh = shapes[part['shape']]()
        mesh.apply_translation(params['position'])
        return mesh

    def _calculate_dimensions(self, part: Dict) -> Dict:
        """Generic proportional reasoning"""
        params = self.default_params.copy()
        size_map = {
            'main_body': {'width': 2.0, 'depth': 1.0, 'height': 0.5},
            'legs': {'radius': 0.05, 'height': 0.7},
            'pillar': {'radius': 0.15, 'height': 1.0},
            'base': {'radius': 0.3, 'height': 0.2}
        }
        
        if part['type'] in size_map:
            params.update(size_map[part['type']])
            self.log_step(f"Set {part['type']} dimensions: {params}")
        
        return params

    def assemble(self) -> trimesh.Trimesh:
        """Generic assembly with spatial reasoning"""
        self.log_step("Assembling components:")
        full_mesh = None
        z_level = 0

        for part in self.components:
            mesh = self.generate_component(part)
            
            # Z-axis stacking logic
            if 'under' in part.get('position', ''):
                z_pos = z_level - mesh.extents[2]/2
                self.log_step(f"Placing {part['type']} under main body at z={z_pos:.2f}")
            else:
                z_level += mesh.extents[2]
                z_pos = z_level
                self.log_step(f"Stacking {part['type']} on top at z={z_pos:.2f}")
                
            mesh.apply_translation([0, 0, z_pos])
            
            full_mesh = mesh if not full_mesh else full_mesh + mesh

        return full_mesh

    def generate(self, description: str) -> trimesh.Trimesh:
        """Full pipeline with visible reasoning"""
        self.reasoning_log = []
        self.parse_description(description)
        mesh = self.assemble()
        
        print("\n".join(self.reasoning_log))
        print(f"\n✅ Generated {len(self.components)} components")
        return mesh

# Example usage
generator = CoT3DGenerator()
mesh = generator.generate("a rectangular table with four legs")
mesh.export("table2.obj")

• Analyzing description: 'a rectangular table with four legs'
• Identified main object type: t
• Detected primary shape: box from keywords
• Detected 4 legs from description
• Assembling components:
• Creating main_body (box)
• Set main_body dimensions: {'width': 2.0, 'depth': 1.0, 'height': 0.5, 'radius': 0.5, 'position': (0, 0, 0)}
• Stacking main_body on top at z=1.00
• Creating legs (cylinder)
• Set legs dimensions: {'width': 1.0, 'depth': 1.0, 'height': 0.7, 'radius': 0.05, 'position': (0, 0, 0)}
• Placing legs under main body at z=0.65

✅ Generated 2 components


'# https://github.com/mikedh/trimesh\nv -0.50000000 -0.50000000 0.50000000\nv -0.50000000 -0.50000000 1.50000000\nv -0.50000000 0.50000000 0.50000000\nv -0.50000000 0.50000000 1.50000000\nv 0.50000000 -0.50000000 0.50000000\nv 0.50000000 -0.50000000 1.50000000\nv 0.50000000 0.50000000 0.50000000\nv 0.50000000 0.50000000 1.50000000\nv 0.00000000 0.00000000 0.30000000\nv 0.05000000 0.00000000 0.30000000\nv 0.05000000 0.00000000 1.00000000\nv 0.00000000 0.00000000 1.00000000\nv 0.04903926 0.00975452 0.30000000\nv 0.04903926 0.00975452 1.00000000\nv 0.04619398 0.01913417 0.30000000\nv 0.04619398 0.01913417 1.00000000\nv 0.04157348 0.02777851 0.30000000\nv 0.04157348 0.02777851 1.00000000\nv 0.03535534 0.03535534 0.30000000\nv 0.03535534 0.03535534 1.00000000\nv 0.02777851 0.04157348 0.30000000\nv 0.02777851 0.04157348 1.00000000\nv 0.01913417 0.04619398 0.30000000\nv 0.01913417 0.04619398 1.00000000\nv 0.00975452 0.04903926 0.30000000\nv 0.00975452 0.04903926 1.00000000\nv 0.00000000 0.050

In [22]:
import trimesh
import numpy as np
from typing import Dict, List

class CoT3DGenerator:
    def __init__(self):
        self.components = []
        self.reasoning_log = []
        self.default_params = {
            'width': 1.0,
            'depth': 1.0,
            'height': 1.0,
            'radius': 0.5,
            'position': (0, 0, 0)
        }

    def log_step(self, message: str):
        """Generic reasoning logger"""
        self.reasoning_log.append(f"• {message}")

    def parse_description(self, text: str):
        """Generic structural decomposition"""
        self.log_step(f"Analyzing description: '{text}'")
        text = text.lower()

        # Generic object type detection
        main_obj = next((w for w in ['table', 'chair', 'lamp', 'shelf'] if w in text), 'object')
        self.log_step(f"Identified main object type: {main_obj[0] if main_obj[0] else 'generic object'}")

        # Shape detection
        shape = self._detect_shape(text)
        self.log_step(f"Detected primary shape: {shape} from keywords")
        
        # Main component
        self.components.append({
            'type': 'main_body',
            'shape': shape,
            'position': 'base'
        })

        # Support detection
        self._detect_supports(text)

    def _detect_shape(self, text: str) -> str:
        """Generic shape reasoning"""
        shape_keywords = {
            'round': 'cylinder',
            'circular': 'cylinder',
            'spherical': 'sphere',
            'rectangular': 'box',
            'angular': 'box',
            'conical': 'cone'
        }
        for kw, shape in shape_keywords.items():
            if kw in text:
                return shape
        return 'box'

    def _detect_supports(self, text: str):
        """Generic support reasoning"""
        support_types = {
            'legs': {'count': 4, 'shape': 'cylinder'},
            'pillar': {'count': 1, 'shape': 'cylinder'},
            'base': {'count': 1, 'shape': 'cylinder'},
            'arms': {'count': 2, 'shape': 'box'}
        }
        
        for kw, config in support_types.items():
            if kw in text:
                count = next((int(n) for n in text.split() if n.isdigit()), config['count'])
                self.log_step(f"Detected {count} {kw} from description")
                self.components.append({
                    'type': kw,
                    'shape': config['shape'],
                    'count': count,
                    'position': 'under_main' if kw == 'legs' else 'base'
                })

    def generate_component(self, part: Dict) -> trimesh.Trimesh:
        """Generic component generation"""
        self.log_step(f"Creating {part['type']} ({part['shape']})")
        params = self._calculate_dimensions(part)
        
        # Shape mapping
        shapes = {
            'box': trimesh.creation.box,
            'cylinder': lambda: trimesh.creation.cylinder(
                radius=params['radius'], height=params['height']),
            'sphere': trimesh.creation.icosphere,
            'cone': lambda: trimesh.creation.cone(
                radius=params['radius'], height=params['height'])
        }
        
        mesh = shapes[part['shape']]()
        mesh.apply_translation(params['position'])
        return mesh

    def _calculate_dimensions(self, part: Dict) -> Dict:
        """Generic proportional reasoning"""
        params = self.default_params.copy()
        size_map = {
            'main_body': {'width': 2.0, 'depth': 1.0, 'height': 0.5},
            'legs': {'radius': 0.05, 'height': 0.7},
            'pillar': {'radius': 0.15, 'height': 1.0},
            'base': {'radius': 0.3, 'height': 0.2}
        }
        
        if part['type'] in size_map:
            params.update(size_map[part['type']])
            self.log_step(f"Set {part['type']} dimensions: {params}")
        
        return params

    def assemble(self) -> trimesh.Trimesh:
        """Generic assembly with spatial reasoning"""
        self.log_step("Assembling components:")
        full_mesh = None
        z_level = 0

        for part in self.components:
            mesh = self.generate_component(part)
            
            # Z-axis stacking logic
            if 'under' in part.get('position', ''):
                z_pos = z_level - mesh.extents[2]/2
                self.log_step(f"Placing {part['type']} under main body at z={z_pos:.2f}")
            else:
                z_level += mesh.extents[2]
                z_pos = z_level
                self.log_step(f"Stacking {part['type']} on top at z={z_pos:.2f}")
                
            mesh.apply_translation([0, 0, z_pos])
            
            full_mesh = mesh if not full_mesh else full_mesh + mesh

        return full_mesh

    def generate(self, description: str) -> trimesh.Trimesh:
        """Full pipeline with visible reasoning"""
        self.reasoning_log = []
        self.parse_description(description)
        mesh = self.assemble()
        
        print("\n".join(self.reasoning_log))
        print(f"\n✅ Generated {len(self.components)} components")
        return mesh

# Example usage
generator = CoT3DGenerator()
mesh = generator.generate("a bulldog with four legs")
mesh.export("table.obj")

• Analyzing description: 'a bulldog with four legs'
• Identified main object type: o
• Detected primary shape: box from keywords
• Detected 4 legs from description
• Assembling components:
• Creating main_body (box)
• Set main_body dimensions: {'width': 2.0, 'depth': 1.0, 'height': 0.5, 'radius': 0.5, 'position': (0, 0, 0)}
• Stacking main_body on top at z=1.00
• Creating legs (cylinder)
• Set legs dimensions: {'width': 1.0, 'depth': 1.0, 'height': 0.7, 'radius': 0.05, 'position': (0, 0, 0)}
• Placing legs under main body at z=0.65

✅ Generated 2 components


'# https://github.com/mikedh/trimesh\nv -0.50000000 -0.50000000 0.50000000\nv -0.50000000 -0.50000000 1.50000000\nv -0.50000000 0.50000000 0.50000000\nv -0.50000000 0.50000000 1.50000000\nv 0.50000000 -0.50000000 0.50000000\nv 0.50000000 -0.50000000 1.50000000\nv 0.50000000 0.50000000 0.50000000\nv 0.50000000 0.50000000 1.50000000\nv 0.00000000 0.00000000 0.30000000\nv 0.05000000 0.00000000 0.30000000\nv 0.05000000 0.00000000 1.00000000\nv 0.00000000 0.00000000 1.00000000\nv 0.04903926 0.00975452 0.30000000\nv 0.04903926 0.00975452 1.00000000\nv 0.04619398 0.01913417 0.30000000\nv 0.04619398 0.01913417 1.00000000\nv 0.04157348 0.02777851 0.30000000\nv 0.04157348 0.02777851 1.00000000\nv 0.03535534 0.03535534 0.30000000\nv 0.03535534 0.03535534 1.00000000\nv 0.02777851 0.04157348 0.30000000\nv 0.02777851 0.04157348 1.00000000\nv 0.01913417 0.04619398 0.30000000\nv 0.01913417 0.04619398 1.00000000\nv 0.00975452 0.04903926 0.30000000\nv 0.00975452 0.04903926 1.00000000\nv 0.00000000 0.050

In [25]:
import trimesh
import numpy as np
from typing import Dict, List

class LayoutPlanner:
    def __init__(self):
        self.rules = {
            'legs': self.arrange_legs,
            'arms': self.arrange_arms,
            'drawers': self.arrange_front_mount
        }

    def arrange_legs(self, main_body, count):
        """Calculate leg positions for tables/chairs"""
        w, d, _ = main_body['dimensions']
        offsets = [
            ( w/2 - 0.1,  d/2 - 0.1, 0),
            (-w/2 + 0.1,  d/2 - 0.1, 0),
            ( w/2 - 0.1, -d/2 + 0.1, 0),
            (-w/2 + 0.1, -d/2 + 0.1, 0)
        ]
        return [offsets[i] for i in range(count)]

    def arrange_arms(self, main_body, count):
        """Side-mounted components"""
        _, d, h = main_body['dimensions']
        return [(0, d/2 + 0.1, h/2)] * count

    def arrange_front_mount(self, main_body, count):
        """Front-facing components"""
        w, _, h = main_body['dimensions']
        return [(w/2 - 0.1*i, 0, h/2) for i in range(count)]

    def plan(self, components):
        """Enhance components with layout data"""
        main_body = next(c for c in components if c['type'] == 'main_body')
        main_body['dimensions'] = [
            main_body.get('width', 1.0),
            main_body.get('depth', 1.0),
            main_body.get('height', 1.0)
        ]

        for part in components:
            if part['type'] in self.rules:
                positions = self.rules[part['type']](
                    main_body, part.get('count', 1))
                part['positions'] = positions
                part['position_strategy'] = 'relative'

        return components

class CoT3DGenerator:
    def __init__(self):
        self.components = []
        self.reasoning_log = []
        self.default_params = {
            'width': 1.0,
            'depth': 1.0,
            'height': 1.0,
            'radius': 0.5,
            'position': (0, 0, 0)
        }

    def log_step(self, message: str):
        self.reasoning_log.append(f"• {message}")

    def parse_description(self, text: str):
        self.log_step(f"Analyzing description: '{text}'")
        text = text.lower()

        main_obj = next((w for w in ['table', 'chair', 'lamp', 'shelf'] if w in text), 'object')
        self.log_step(f"Identified main object type: {main_obj}")

        shape = self._detect_shape(text)
        self.log_step(f"Detected primary shape: {shape}")
        
        self.components.append({
            'type': 'main_body',
            'shape': shape,
            'position': 'base'
        })

        self._detect_supports(text)

    def _detect_shape(self, text: str) -> str:
        shape_keywords = {
            'round': 'cylinder',
            'circular': 'cylinder',
            'spherical': 'sphere',
            'rectangular': 'box',
            'angular': 'box',
            'conical': 'cone'
        }
        for kw, shape in shape_keywords.items():
            if kw in text:
                return shape
        return 'box'

    def _detect_supports(self, text: str):
        support_types = {
            'legs': {'count': 4, 'shape': 'cylinder'},
            'pillar': {'count': 1, 'shape': 'cylinder'},
            'base': {'count': 1, 'shape': 'cylinder'},
            'arms': {'count': 2, 'shape': 'box'}
        }
        
        for kw, config in support_types.items():
            if kw in text:
                count = next((int(n) for n in text.split() if n.isdigit()), config['count'])
                self.log_step(f"Detected {count} {kw} from description")
                self.components.append({
                    'type': kw,
                    'shape': config['shape'],
                    'count': count,
                    'position': 'under_main' if kw == 'legs' else 'base'
                })

    def generate_component(self, part: Dict) -> trimesh.Trimesh:
        self.log_step(f"Creating {part['type']} ({part['shape']})")
        params = self._calculate_dimensions(part)
        
        shapes = {
            'box': trimesh.creation.box,
            'cylinder': lambda: trimesh.creation.cylinder(
                radius=params['radius'], height=params['height']),
            'sphere': trimesh.creation.icosphere,
            'cone': lambda: trimesh.creation.cone(
                radius=params['radius'], height=params['height'])
        }
        
        mesh = shapes[part['shape']]()
        mesh.apply_translation(params['position'])
        return mesh

    def _calculate_dimensions(self, part: Dict) -> Dict:
        params = self.default_params.copy()
        size_map = {
            'main_body': {'width': 2.0, 'depth': 1.0, 'height': 0.5},
            'legs': {'radius': 0.05, 'height': 0.7},
            'pillar': {'radius': 0.15, 'height': 1.0},
            'base': {'radius': 0.3, 'height': 0.2}
        }
        
        if part['type'] in size_map:
            params.update(size_map[part['type']])
            self.log_step(f"Set {part['type']} dimensions: {params}")
        
        return params

    def assemble(self) -> trimesh.Trimesh:
        full_mesh = None
        z_level = 0

        for part in self.components:
            mesh = self.generate_component(part)
            
            if 'under' in part.get('position', ''):
                z_pos = z_level - mesh.extents[2]/2
                self.log_step(f"Placing {part['type']} under main body at z={z_pos:.2f}")
            else:
                z_level += mesh.extents[2]
                z_pos = z_level
                self.log_step(f"Stacking {part['type']} on top at z={z_pos:.2f}")
                
            mesh.apply_translation([0, 0, z_pos])
            full_mesh = mesh if not full_mesh else full_mesh + mesh

        return full_mesh

    def generate(self, description: str) -> trimesh.Trimesh:
        self.reasoning_log = []
        self.parse_description(description)
        mesh = self.assemble()
        
        print("\n".join(self.reasoning_log))
        print(f"\n✅ Generated {len(self.components)} components")
        return mesh

class EnhancedCoT3DGenerator(CoT3DGenerator):
    def __init__(self):
        super().__init__()
        self.layout_planner = LayoutPlanner()
        self.default_params.update({
            'drawer_width': 0.3,
            'taper_ratio': 0.7
        })
        
    def parse_description(self, text: str):
        super().parse_description(text)
        
        if 'drawer' in text:
            self.log_step("Detected storage component")
            self.components.append({
                'type': 'drawer',
                'shape': 'box',
                'count': 1,
                'position': 'front'
            })
            
        if 'tapered' in text:
            self.log_step("Detected tapered geometry")
            for c in self.components:
                if c['type'] == 'legs':
                    c['shape'] = 'tapered_cylinder'

    def assemble(self) -> trimesh.Trimesh:
        self.log_step("Running spatial planning...")
        self.components = self.layout_planner.plan(self.components)
        
        full_mesh = None
        for part in self.components:
            if part.get('count', 1) > 1:
                for i in range(part['count']):
                    mesh = self.generate_component(part, instance_idx=i)
                    if part.get('position_strategy') == 'relative':
                        x, y, z = part['positions'][i]
                        mesh.apply_translation([x, y, z])
                    full_mesh = mesh if not full_mesh else full_mesh + mesh
            else:
                mesh = self.generate_component(part)
                if part.get('position_strategy') == 'relative':
                    x, y, z = part.get('positions', [(0,0,0)])[0]
                    mesh.apply_translation([x, y, z])
                full_mesh = mesh if not full_mesh else full_mesh + mesh

        return full_mesh

    def generate_component(self, part: Dict, instance_idx=0) -> trimesh.Trimesh:
        self.log_step(f"Creating {part['type']} ({part['shape']})")
        params = self._calculate_dimensions(part)
        
        if part['shape'] == 'tapered_cylinder':
            radius_top = params['radius'] * self.default_params['taper_ratio']
            mesh = trimesh.creation.cylinder(
                radius=params['radius'],
                height=params['height'],
                sections=32,
                transform=trimesh.transformations.taper_transform(
                    radius_top/params['radius']))
        else:
            shapes = {
                'box': trimesh.creation.box,
                'cylinder': lambda: trimesh.creation.cylinder(
                    radius=params['radius'], height=params['height']),
                'sphere': trimesh.creation.icosphere,
                'cone': lambda: trimesh.creation.cone(
                    radius=params['radius'], height=params['height'])
            }
            mesh = shapes[part['shape']]()
        
        return mesh

# Example usage
generator = EnhancedCoT3DGenerator()
mesh = generator.generate(
    "An oak desk with two tapered drawers and four carved legs")
mesh.export("complex_desk.obj")

AttributeError: module 'trimesh.transformations' has no attribute 'taper_transform'

In [26]:
import trimesh
import numpy as np
from typing import Dict, List
from transformers import pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC

class MLIntentClassifier:
    def __init__(self):
        # Simple ML model for demonstration (replace with actual trained model)
        self.vectorizer = TfidfVectorizer()
        self.classifier = LinearSVC()
        self.intents = ['furniture', 'lighting', 'storage', 'decorative']
        
        # Mock training data (replace with real dataset)
        mock_data = [
            ("table with legs", 'furniture'),
            ("lamp with shade", 'lighting'),
            ("ornamental sculpture", 'decorative'),
            ("bookshelf with drawers", 'storage')
        ]
        texts, labels = zip(*mock_data)
        X = self.vectorizer.fit_transform(texts)
        self.classifier.fit(X, labels)

    def predict(self, text: str) -> str:
        X = self.vectorizer.transform([text])
        return self.classifier.predict(X)[0]

class ShapeGenerator:
    @staticmethod
    def create_shape(shape_type: str, params: Dict) -> trimesh.Trimesh:
        shapes = {
            'box': lambda: trimesh.creation.box([params['width'], params['depth'], params['height']]),
            'cylinder': lambda: trimesh.creation.cylinder(
                radius=params['radius'], 
                height=params['height'],
                sections=params.get('sections', 32)
            ),
            'torus': lambda: trimesh.creation.torus(
                radius=params['radius'],
                section_radius=params['section_radius']
            ),
            'pyramid': lambda: trimesh.creation.cone(
                radius=params['base_radius'],
                height=params['height'],
                sections=4
            ),
            'prism': lambda: trimesh.creation.cylinder(
                radius=params['radius'],
                height=params['height'],
                sections=params['sides']
            ),
            'geodesic': lambda: trimesh.creation.icosphere(
                radius=params['radius'],
                subdivisions=2
            ),
            'helix': lambda: ShapeGenerator.create_helix(
                turns=params['turns'],
                radius=params['radius'],
                height=params['height']
            )
        }
        return shapes[shape_type]()

    @staticmethod
    def create_helix(turns: int, radius: float, height: float) -> trimesh.Trimesh:
        theta = np.linspace(0, 2*np.pi*turns, 100*turns)
        z = np.linspace(0, height, len(theta))
        points = np.vstack([radius*np.cos(theta), radius*np.sin(theta), z]).T
        return trimesh.load_path(points).cylinder(radius=0.02)

class EnhancedCoT3DGenerator:
    def __init__(self):
        self.components = []
        self.reasoning_log = []
        self.intent_classifier = MLIntentClassifier()
        self.layout_planner = LayoutPlanner()
        self.shape_generator = ShapeGenerator()
        
        self.default_params = {
            'width': 1.0,
            'depth': 1.0,
            'height': 1.0,
            'radius': 0.5,
            'section_radius': 0.1,
            'sides': 6,
            'turns': 3,
            'base_radius': 0.5
        }

    def log_step(self, message: str):
        self.reasoning_log.append(f"• {message}")

    def parse_description(self, text: str):
        self.log_step(f"Analyzing description: '{text}'")
        text = text.lower()
        
        # ML Intent Classification
        intent = self.intent_classifier.predict(text)
        self.log_step(f"ML-classified intent: {intent.upper()}")
        
        # Intent-based configuration
        if intent == 'furniture':
            self.default_params.update({'width': 2.0, 'height': 0.8})
        elif intent == 'lighting':
            self.default_params.update({'radius': 0.3, 'height': 1.5})

        # Shape detection with extended vocabulary
        shape = self._detect_shape(text)
        self.log_step(f"Detected primary shape: {shape}")
        
        self.components.append({
            'type': 'main_body',
            'shape': shape,
            'position': 'base'
        })

        self._detect_supports(text)
        self._detect_special_features(text)

    def _detect_shape(self, text: str) -> str:
        shape_keywords = {
            'torus': 'torus',
            'helical': 'helix',
            'spiral': 'helix',
            'pyramid': 'pyramid',
            'prism': 'prism',
            'hexagonal': 'prism',
            'geodesic': 'geodesic',
            'dome': 'geodesic',
            'round': 'cylinder',
            'angular': 'box'
        }
        for kw, shape in shape_keywords.items():
            if kw in text:
                return shape
        return 'box'

    def _detect_supports(self, text: str):
        support_types = {
            'legs': {'shape': 'cylinder', 'params': {'radius': 0.05}},
            'pillar': {'shape': 'prism', 'params': {'sides': 8}},
            'base': {'shape': 'torus', 'params': {'section_radius': 0.02}},
            'arms': {'shape': 'box', 'params': {}}
        }
        
        for kw, config in support_types.items():
            if kw in text:
                count = next((int(n) for n in text.split() if n.isdigit()), 4)
                self.log_step(f"Detected {count} {kw} using {config['shape']} shape")
                self.components.append({
                    'type': kw,
                    'shape': config['shape'],
                    'count': count,
                    'params': config['params'],
                    'position': 'under_main' if kw == 'legs' else 'base'
                })

    def _detect_special_features(self, text: str):
        if 'twisted' in text:
            self.log_step("Adding twist deformation")
            self.components[-1]['deformation'] = 'twist'
            
        if 'textured' in text:
            self.log_step("Enabling surface patterning")
            self.components[-1]['texture'] = True

    def generate_component(self, part: Dict) -> trimesh.Trimesh:
        self.log_step(f"Generating {part['type']} ({part['shape']})")
        params = self._calculate_dimensions(part)
        
        # Apply shape-specific parameters
        params.update(part.get('params', {}))
        
        mesh = self.shape_generator.create_shape(part['shape'], params)
        
        # Apply deformations
        if 'deformation' in part:
            mesh = self._apply_deformation(mesh, part['deformation'])
            
        return mesh

    def _apply_deformation(self, mesh: trimesh.Trimesh, deformation: str) -> trimesh.Trimesh:
        if deformation == 'twist':
            matrix = trimesh.transformations.twist_matrix(
                np.pi/2,  # 90 degree twist
                mesh.bounds[0][2],  # Start at bottom
                mesh.bounds[1][2]   # End at top
            )
            mesh.apply_transform(matrix)
        return mesh

    def assemble(self) -> trimesh.Trimesh:
        self.log_step("Running spatial planning...")
        self.components = self.layout_planner.plan(self.components)
        
        full_mesh = None
        for part in self.components:
            if part.get('count', 1) > 1:
                for i in range(part['count']):
                    mesh = self.generate_component(part)
                    if 'positions' in part:
                        mesh.apply_translation(part['positions'][i])
                    full_mesh = mesh if not full_mesh else full_mesh + mesh
            else:
                mesh = self.generate_component(part)
                full_mesh = mesh if not full_mesh else full_mesh + mesh

        return full_mesh

    def generate(self, description: str) -> trimesh.Trimesh:
        self.components = []
        self.reasoning_log = []
        
        self.parse_description(description)
        mesh = self.assemble()
        
        print("\n".join(self.reasoning_log))
        print(f"\n✅ Generated {len(self.components)} components")
        return mesh

# Example usage
generator = EnhancedCoT3DGenerator()
complex_mesh = generator.generate(
    "A modern lamp with helical base and four twisted metal legs")
complex_mesh.export("art_lamp.obj")

AttributeError: 'EnhancedCoT3DGenerator' object has no attribute '_calculate_dimensions'